In [1]:
import sys, numpy as np

In [2]:
from keras.datasets import mnist

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 11s 1us/step


In [7]:
images, labels = (x_train[0:1000].reshape(1000,28*28)/255, y_train[0:1000])

In [11]:
one_hot_labels = np.zeros((len(labels),10))

for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

In [12]:
test_images = x_test.reshape(len(x_test),28*28)/255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

In [13]:
np.random.seed(1)

In [14]:
relu = lambda x:(x>=0)*x
relu2deriv = lambda x: x>=0

In [15]:
alpha = 0.005
iterations = 350
hidden_size = 40
pixels_per_image = 784
num_labels = 10

In [16]:
weights_0_1 = 0.2 * np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

In [17]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        error += np.sum((labels[i:i+1] - layer_2)**2)
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))
        
        layer_2_delta = (layer_2 - labels[i:i+1])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        
        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 -= alpha * layer_0.T.dot(layer_1_delta)
        
    sys.stdout.write('\r'+' I:'+str(j)+' Error:'+str(error/float(len(images)))[0:5]+' Correct:'+str(correct_cnt/float(len(images))))

 I:349 Error:0.108 Correct:1.099

In [18]:
if (j%10 == 0 or j == iterations -1):
    error, correct_cnt = 0.0, 0
    
    for i in range(len(test_images)):
        
        layer_0 = test_images[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)
        
        error += (np.sum((layer_2 - test_labels[i:i+1]) **2))
        correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
    sys.stdout.write(' Test-Err: ' +str(error/float(len(test_images)))[0:5] + 
                     ' Test-Acc: ' + str(correct_cnt/float(len(test_images))))
    print()

 Test-Err: 0.653 Test-Acc: 0.7073


In [19]:
import numpy as np
import sys

np.random.seed(1)

from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28)/255, y_train[0:1000])

one_hot_labels = np.zeros((len(labels),10))
for i,j in enumerate(labels):
    one_hot_labels[i][j] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28)/255
test_labels = np.zeros((len(y_test),10))

for i,j in enumerate(y_test):
    test_labels[i][j] = 1
    
def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1-(output**2)

def softmax(x):
    temp = np.exp(x)
    return temp/np.sum(temp, axis=1, keepdims=True)

alpha, iterations, hidden_size = 2,300,100

pixels_per_image, num_labels = 784, 10

batch_size = 100

weights_0_1 = 0.02*np.random.random((pixels_per_image,hidden_size)) - 0.01
weights_1_2 = 0.02*np.random.random((hidden_size, num_labels)) - 0.01

for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images)/batch_size)):
        batch_start, batch_end = i*batch_size,(i+1)*batch_size
        layer_0 = images[batch_start:batch_end]
        layer_1 = tanh(np.dot(layer_0, weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask*2
        layer_2 = softmax(np.dot(layer_1, weights_1_2))
        
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))
        layer_2_delta = (layer_2 - labels[batch_start:batch_end]) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        
        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 -= alpha * layer_0.T.dot(layer_1_delta)
        
    test_correct_cnt = 0
    
    for i in range(len(test_images)):
        layer_0 = test_images[i:i+1]
        layer_1 = tanh(np.dot(layer_0,weights_0_1))
        layer_2 = softmax(np.dot(layer_1, weights_1_2))
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))
        
    if (j%10==0):
        sys.stdout.write('\n'+'I:'+str(j)+
                         'Test-Acc' + str(test_correct_cnt/float(len(test_images)))+
                         'Train-Acc' + str(correct_cnt/float(len(images))))


I:0Test-Acc0.1422Train-Acc0.104
I:10Test-Acc0.6065Train-Acc0.564
I:20Test-Acc0.5836Train-Acc0.6
I:30Test-Acc0.5389Train-Acc0.561
I:40Test-Acc0.5259Train-Acc0.568
I:50Test-Acc0.5632Train-Acc0.585
I:60Test-Acc0.6149Train-Acc0.642
I:70Test-Acc0.6711Train-Acc0.688
I:80Test-Acc0.7189Train-Acc0.731
I:90Test-Acc0.7606Train-Acc0.778
I:100Test-Acc0.7831Train-Acc0.818
I:110Test-Acc0.802Train-Acc0.839
I:120Test-Acc0.8112Train-Acc0.853
I:130Test-Acc0.8177Train-Acc0.873
I:140Test-Acc0.8229Train-Acc0.871
I:150Test-Acc0.8264Train-Acc0.875
I:160Test-Acc0.8324Train-Acc0.882
I:170Test-Acc0.8355Train-Acc0.897
I:180Test-Acc0.8395Train-Acc0.899
I:190Test-Acc0.8433Train-Acc0.898
I:200Test-Acc0.8466Train-Acc0.912
I:210Test-Acc0.8494Train-Acc0.906
I:220Test-Acc0.8506Train-Acc0.917
I:230Test-Acc0.8525Train-Acc0.917
I:240Test-Acc0.8542Train-Acc0.918
I:250Test-Acc0.856Train-Acc0.926
I:260Test-Acc0.8573Train-Acc0.929
I:270Test-Acc0.8584Train-Acc0.933
I:280Test-Acc0.8589Train-Acc0.933
I:290Test-Acc0.8592Train-Acc